# Modulos

In [1]:
import json
import time
import os
from libbgg.apiv1 import BGG
import pandas as pd

# Relevant functions

In [2]:
def baseGame(idGame,expansion, conexion):
    print("entre a baseGame")
    if(idGame in expansion):
        expansion.add(idGame)
        game=0
        return (expansion,False,game)
    else:
        game=conexion.get_game(int(idGame))
        if('boardgamecategory' in game["boardgames"]["boardgame"].keys()):
            ls_cat= game["boardgames"]["boardgame"]['boardgamecategory']
            if(type(ls_cat) is list):
                for category in ls_cat:
                    if(category["objectid"]=='1042'):
                        print("Es una expansion")
                        expansion.add(idGame)
                        return (expansion,False,game)
            else:
                if(game["boardgames"]["boardgame"]['boardgamecategory']["objectid"]=='1042'):
                    expansion.add(idGame)
                    return (expansion,False,game)
    return(expansion,True,game)

In [3]:
def getData(username,collection,expansion,storedGames,conexion):
    ls_ids= []
    ls_ids_game=[]
    ls_names_game=[]
    ls_userR=[]
    ls_avgR=[]
    ls_status= []
    ls_names=[]
    ls_descriptions=[]
    dc_game= {}
    dc_user={}
    for item in range(len(collection["items"]["item"])):
        print(f'Estamos en el item {item}------------------------------')
        userOwner =collection["items"]["item"][item]["status"]["own"]
        userWant = collection["items"]["item"][item]["status"]['wanttoplay']
        if(userOwner== '1' or userWant=='1'):
            print("Todo bien en owner")
            gameId=collection["items"]["item"][item]["objectid"]
            base=1
            if(gameId not in storedGames):
                print("No esta guardado")
                expansion,base,game= baseGame(gameId,expansion,conexion)
                if(base):
                    print("Es un juego base")
                    gameName= collection["items"]["item"][item]["name"]["TEXT"]
                    userRating= collection["items"]["item"][item]["stats"]["rating"]["value"]
                    avgRating= collection["items"]["item"][item]["stats"]["rating"]["average"]["value"]
                    description= game["boardgames"]["boardgame"]["description"]["TEXT"]
                    storedGames.add(gameId)
                    ls_ids_game.append(gameId)
                    ls_ids.append(gameId)
                    ls_names.append(gameName)
                    ls_names_game.append(gameName)
                    ls_avgR.append(avgRating)
                    ls_status.append(userOwner)
                    ls_descriptions.append(description)
                    ls_userR.append(userRating)
            else:
                print("Ya esta guardado")
                gameName= collection["items"]["item"][item]["name"]["TEXT"]
                userRating= collection["items"]["item"][item]["stats"]["rating"]["value"]
                avgRating= collection["items"]["item"][item]["stats"]["rating"]["average"]["value"]
                ls_ids.append(gameId)
                ls_names.append(gameName)
                ls_avgR.append(avgRating)
                ls_status.append(userOwner)
                ls_userR.append(userRating)
    dc_user["UserName"]= [username for x in range(len(ls_ids))]
    dc_user["GameId"]= ls_ids
    dc_user["GameName"]= ls_names
    dc_user["Status"]= ls_status
    dc_user["UserRanking"]= ls_userR
    dc_user["Ranking"]= ls_avgR
    dc_game["GameId"]= ls_ids_game
    dc_game["GameName"]= ls_names_game
    dc_game["Description"]= ls_descriptions
    dataUser=pd.DataFrame(dc_user)
    dataGames= pd.DataFrame(dc_game)
    
    return dataUser,dataGames,expansion,storedGames

# API conexion

In [4]:
conn = BGG()

# Collect data

## Get User's collection

In [46]:
ls_users=["Ramssesmex", "aleck22", "aloartegrafico","benthor","Blacklira","CesarcruCed","cova88","crossworld","dianacronica",
         "Dieggo","dieterminator","dvtlpzrc","Earth_Worm_Jim","EcatlOcelotl","EddyVaffen", "fertorresmx", "fleshman19", "gandalfgrismx",
         "Giovash", "gorgue","johnnydarko4550", "JorgeReno", "KaeruGames","Kakiri", "Laudyne", "Liboo", "Luis Aguirre", "malakav", "marianoth" ]

In [6]:
len(ls_users)

30

In [14]:
for user in ls_users:
    try:
        collection=conn.get_collection(username=user)
    except:
        print(f'Fallo el request en {user} estamos esperando dos minutos.')
        time.sleep(120)
        collection=conn.get_collection(username=user)
    json_object= json.dumps(collection,indent=4)
    jsonName='UsersJson/'+ user+'.json'
    with open(jsonName, "w") as outfile:
        outfile.write(json_object)

Fallo el request en cova88 estamos esperando dos minutos.
Fallo el request en gorgue estamos esperando dos minutos.
Fallo el request en Kakiri estamos esperando dos minutos.
Fallo el request en marianoth estamos esperando dos minutos.


## Create user and boardgame table

In [51]:
data_boardgames=pd.DataFrame(columns=['GameId', 'GameName', 'Description'])
data_users=pd.DataFrame(columns=['UserName', 'GameId', 'GameName', 'Status', 'UserRanking', 'Ranking'])
storedGames=set()
expansion=set()

In [52]:
directory= "UsersJson"
paths= []
for filename in os.listdir(directory):
    f= os.path.join(directory,filename)
    paths.append(f)

In [53]:
paths.remove('UsersJson/.ipynb_checkpoints')

In [54]:
for path in paths:
    with open(path, 'r') as openfile:
        # Reading from json file
        collection = json.load(openfile)
    userName= path.split("/")[1].split(".")[0]
    dataUser,dataGames,expansion,storedGames =getData(userName,collection,expansion,storedGames,conn)
    data_boardgames=pd.concat([data_boardgames,dataGames], ignore_index=True)
    data_users=pd.concat([data_users,dataUser], ignore_index=True)

Estamos en el item 0------------------------------
Todo bien en owner
No esta guardado
entre a baseGame
Es un juego base
Estamos en el item 1------------------------------
Todo bien en owner
No esta guardado
entre a baseGame
Es un juego base
Estamos en el item 2------------------------------
Todo bien en owner
No esta guardado
entre a baseGame
Es un juego base
Estamos en el item 3------------------------------
Todo bien en owner
No esta guardado
entre a baseGame
Es una expansion
Estamos en el item 4------------------------------
Todo bien en owner
No esta guardado
entre a baseGame
Es un juego base
Estamos en el item 5------------------------------
Todo bien en owner
No esta guardado
entre a baseGame
Es un juego base
Estamos en el item 6------------------------------
Todo bien en owner
No esta guardado
entre a baseGame
Es una expansion
Estamos en el item 7------------------------------
Todo bien en owner
No esta guardado
entre a baseGame
Es un juego base
Estamos en el item 8------------

In [55]:
data_boardgames["GameId"].value_counts()

148494    1
258210    1
180602    1
302723    1
223040    1
         ..
171663    1
155068    1
285967    1
181495    1
248065    1
Name: GameId, Length: 2143, dtype: int64

In [56]:
data_users["UserName"].value_counts()

Ramssesmex         496
aleck22            465
Kakiri             397
KaeruGames         281
cova88             267
marianoth          231
dieterminator      199
Luis Aguirre       198
Giovash            185
Liboo              182
Earth_Worm_Jim     168
CesarcruCed        163
aloartegrafico     158
johnnydarko4550    142
gandalfgrismx      139
Blacklira          113
malakav             97
dvtlpzrc            80
benthor             73
fertorresmx         56
crossworld          55
fleshman19          52
JorgeReno           49
EcatlOcelotl        46
gorgue              41
dianacronica        39
Dieggo              36
EddyVaffen          11
Laudyne              5
Name: UserName, dtype: int64

In [58]:
data_users[["UserName","GameName"]].value_counts()

UserName   GameName                
Kakiri     Maratón Clásico             5
           Súper Maratón               3
           Outside the Scope of BGG    3
           Memory                      3
Liboo      Similo                      2
                                      ..
           Hanamikoji                  1
           Happy Little Dinosaurs      1
           Hero Realms                 1
           High Society                1
marianoth  ロイヤルターフ                     1
Length: 4386, dtype: int64

In [49]:
data_users.to_csv("users.csv", index=False)
data_boardgames.to_csv("games.csv", index= False)

In [60]:
data_users[data_users["GameName"]=='Maratón Clásico']

,UserName,GameId,GameName,Status,UserRanking,Ranking
521,Kakiri,26033,Maratón Clásico,1,7.449,6.49244
522,Kakiri,26033,Maratón Clásico,1,7.479,6.49244
523,Kakiri,26033,Maratón Clásico,1,7.319,6.49244
524,Kakiri,26033,Maratón Clásico,1,6.759,6.49244
525,Kakiri,26033,Maratón Clásico,1,N/A,6.49244
714,EcatlOcelotl,26033,Maratón Clásico,1,N/A,6.49244
2464,dieterminator,26033,Maratón Clásico,1,7,6.49244
3761,fertorresmx,26033,Maratón Clásico,1,N/A,6.49244


In [28]:
dc_user["UserName"]= ["Ramssesmex" for x in range(len(ls_ids))]
dc_user["GameId"]= ls_ids
dc_user["GameName"]= ls_names
dc_user["Status"]= ls_status
dc_user["UserRanking"]= ls_userR
dc_user["Ranking"]= ls_avgR
dc_game["GameId"]= ls_ids
dc_game["GameName"]= ls_names
dc_game["Description"]= ls_descriptions

In [29]:
dataGames= pd.DataFrame(dc_game)

In [30]:
dataUser=pd.DataFrame(dc_user)

In [31]:
dataUser

,UserName,GameId,GameName,Status,UserRanking,Ranking
0,Ramssesmex,17405,1846: The Race for the Midwest,0,N/A,7.9511
1,Ramssesmex,267058,300: Earth & Water,1,N/A,7.50619
2,Ramssesmex,268586,6 nimmt! 25 Jahre,1,7.5,7.42317
3,Ramssesmex,68448,7 Wonders,1,7.3,7.71387
4,Ramssesmex,173346,7 Wonders Duel,1,7.7,8.10053
...,...,...,...,...,...,...
491,Ramssesmex,82222,Xia: Legends of a Drift System,1,N/A,7.87664
492,Ramssesmex,117915,Yedo,1,N/A,7.37105
493,Ramssesmex,280575,Yin Yang,1,N/A,7.30356
494,Ramssesmex,196340,Yokohama,1,8.1,7.82919


In [32]:
dataGames.head(5)

,GameId,GameName,Description
0,17405,1846: The Race for the Midwest,1846 is an 18xx game that traces the westward ...
1,267058,300: Earth & Water,"In 500 BC, Ionia rebelled against Persia's rul..."
2,268586,6 nimmt! 25 Jahre,"In 6 nimmt!, you want to score as few points a..."
3,68448,7 Wonders,You are the leader of one of the 7 great citie...
4,173346,7 Wonders Duel,In many ways 7 Wonders Duel resembles its pare...
